# 주소 => 위도와 경도 


In [1]:
# 주피터 노트북 환경설정
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

from IPython.display import Image

from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 100% !important; }</style>"))

# 카카오 API 키 발급

- https://developers.kakao.com/ 로그인 
- [내 애플리케이션]-[애플리케이션 추가하기]
- 생성된 앱 클릭 후 왼쪽 메뉴에서 [플랫폼] 클릭
- Web 플랫폼 등록 
- [사이트 도메인 주소]추가 
    - http://localhost:8080
    - http://localhost:8890
    - http://localhost:8888
- 앱 다시 선택 
    : 발급받은 APT 키중에서 REST API 키 복사

In [23]:
# 라이브러리 추가
import requests; 
from urllib.parse import urlparse
import pandas as pd
import json

In [3]:
# # 카카오맵 주소 검색 API: https://developers.kakao.com/docs/restapi/local#주소-검색
# address = "제주 서귀포시 서홍동 447-3"
# url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address

# result = requests.get(urlparse(url).geturl(),
#                       headers={"Authorization":"KakaoAK <REST API 키>})
# json_obj = result.json()
# print(json_obj)

In [4]:
# 카카오맵 주소 검색 API: https://developers.kakao.com/docs/restapi/local#주소-검색
address = "강남구 언주로 508"
url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address

result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization":"KakaoAK 4b7bb12227be8c980dc43f0a0b939865"})
json_obj = result.json()
print(json_obj)

{'documents': [{'address': {'address_name': '서울 강남구 역삼동 701', 'b_code': '1168010100', 'h_code': '1168064000', 'main_address_no': '701', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_h_name': '역삼1동', 'region_3depth_name': '역삼동', 'sub_address_no': '', 'x': '127.042909941112', 'y': '37.5039660612952'}, 'address_name': '서울 강남구 언주로 508', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 강남구 언주로 508', 'building_name': '공무원연금공단', 'main_building_no': '508', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_name': '역삼동', 'road_name': '언주로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '127.042909941112', 'y': '37.5039660612952', 'zone_no': '06152'}, 'x': '127.042909941112', 'y': '37.5039660612952'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}


In [27]:
# 카카오맵 주소 검색 API: https://developers.kakao.com/docs/restapi/local#주소-검색
address = "강남구 언주로 508"
url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address

result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization":"KakaoAK 4b7bb12227be8c980dc43f0a0b939865"})
# json_obj = result.json()
json_obj = json.loads(result.text)
print(json_obj)

{'documents': [{'address': {'address_name': '서울 강남구 역삼동 701', 'b_code': '1168010100', 'h_code': '1168064000', 'main_address_no': '701', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_h_name': '역삼1동', 'region_3depth_name': '역삼동', 'sub_address_no': '', 'x': '127.042909941112', 'y': '37.5039660612952'}, 'address_name': '서울 강남구 언주로 508', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 강남구 언주로 508', 'building_name': '공무원연금공단', 'main_building_no': '508', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_name': '역삼동', 'road_name': '언주로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '127.042909941112', 'y': '37.5039660612952', 'zone_no': '06152'}, 'x': '127.042909941112', 'y': '37.5039660612952'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}


In [28]:
# 검색 결과 정렬
list = []
for document in json_obj['documents']:
    val = [document['road_address']['building_name'], 
               document['address_name'], document['y'], document['x']]
    list.append(val)
    
df = pd.DataFrame(list, columns = ['building_name', 'address_name', 'lat', 'lon'])



In [29]:
df

,building_name,address_name,lat,lon
0,공무원연금공단,서울 강남구 언주로 508,37.5039660612952,127.042909941112


# folium 시각화

In [7]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#  한글글꼴로 변경
plt.rcParams['font.size'] = 12.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'
print(plt.rcParams['font.family'])
print(plt.rcParams['font.size'])
import folium

['Malgun Gothic']
12.0


# 제주도 선별 진료소

In [8]:
df_clinic =pd.read_csv('data/clinic_total.csv', 
                       encoding='utf-8', index_col=2)

In [9]:
df_clinic.head()

,시도,시군구,주소,평일 운영시간,토요일 운영시간,일요일/공휴일\n운영시간,대표 전화번호
의료기관명,,,,,,,
강남구보건소,서울,강남구,서울 강남구 삼성동(삼성2동) 8 강남구보건소,09:00 ~ 20:00,09:00 ~ 19:00,09:00 ~ 19:00,02-3423-5555
삼성서울병원,서울,강남구,서울 강남구 일원로81 삼성서울병원,00:00 ~ 24:00,00:00 ~ 24:00,00:00 ~ 24:00,02-3410-0543
강남세브란스병원,서울,강남구,서울 강남구 언주로211 강남세브란스병원,09:00 ~ 17:00,09:00 ~ 12:30,미운영,02-1599-6114
강동구보건소,서울,강동구,서울 강동구 성내동 541-2,09:00 ~ 18:00,09:00 ~ 16:00,09:00 ~ 16:00,02-3425-6713
중앙보훈병원,서울,강동구,서울 강동구 진황도로 61길 53,09:00 ~ 17:30,미운영,미운영,02-2225-1114


In [10]:
df_clinic = df_clinic[df_clinic['시도']=='제주']
df_clinic.shape


(11, 7)

In [11]:
df_clinic

,시도,시군구,주소,평일 운영시간,토요일 운영시간,일요일/공휴일\n운영시간,대표 전화번호
의료기관명,,,,,,,
서귀포시(서귀포보건소),제주,서귀포시,제주 서귀포시 서홍동 447-3,09:00 ~ 18:00,미운영,미운영,064-760-6092
서귀포열린병원,제주,서귀포시,제주특별자치도 서귀포시 일주동로 8638,09:00 ~ 18:00,09:00 ~ 11:00,미운영,064-762-8001
서귀포의료원,제주,서귀포시,제주특별자치도 서귀포시 장수로 47,00:00 ~ 24:00,00:00 ~ 24:00,00:00 ~ 24:00,064-730-3518
서귀포시(서부보건소),제주,서귀포시,제주 서귀포시 대정읍 하모리 767-8 서부보건소,09:00 ~ 16:00,09:00 ~ 16:00,09:00 ~ 16:00,064-760-6292
제주시제주보건소,제주,제주시,제주 제주시 도남동 567-1,09:00 ~ 18:00,미운영,미운영,064-728-1410
제주한라병원,제주,제주시,도령로 65,00:00 ~ 24:00,00:00 ~ 24:00,00:00 ~ 24:00,064-740-5158
한마음병원,제주,제주시,연신로 52,00:00 ~ 24:00,00:00 ~ 24:00,00:00 ~ 24:00,064-750-9119
한국병원,제주,제주시,서광로 193,00:00 ~ 24:00,00:00 ~ 24:00,00:00 ~ 24:00,064-750-0119
중앙병원,제주,제주시,월랑로 91,00:00 ~ 24:00,00:00 ~ 24:00,00:00 ~ 24:00,064-786-7119


In [12]:
df_clinic = df_clinic[['시군구','주소']]
df_clinic.head()

,시군구,주소
의료기관명,,
서귀포시(서귀포보건소),서귀포시,제주 서귀포시 서홍동 447-3
서귀포열린병원,서귀포시,제주특별자치도 서귀포시 일주동로 8638
서귀포의료원,서귀포시,제주특별자치도 서귀포시 장수로 47
서귀포시(서부보건소),서귀포시,제주 서귀포시 대정읍 하모리 767-8 서부보건소
제주시제주보건소,제주시,제주 제주시 도남동 567-1


In [30]:
lat=[]
long=[]
for i in range(len(df_clinic.index)):
    address=df_clinic.iloc[i,1]
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" \
            + address
    result = requests.get(urlparse(url).geturl(),
             headers={"Authorization": \
             "KakaoAK 4b7bb12227be8c980dc43f0a0b939865"})
    # json_obj = result.json()
    json_obj = json.loads(result.text)
    
    for document in json_obj['documents']:
        lat.append(document['y'])
        long.append(document['x'])
        print(address)
        print(lat[i])
        print(long[i])

제주 서귀포시 서홍동 447-3
33.2532700888151
126.555250836738
제주특별자치도 서귀포시 일주동로 8638
33.2544659162544
126.565425372675
제주특별자치도 서귀포시 장수로 47 
33.2559809912099
126.562948872895
제주 서귀포시 대정읍 하모리 767-8 서부보건소
33.2186942969756
126.253410446884
제주 제주시 도남동  567-1
33.4917394006488
126.520234177096
도령로 65
33.4902661749243
126.484919722369
연신로 52
33.4962868079834
126.546238571767
서광로 193
33.5003915262127
126.516729196278
월랑로 91
33.4923211704292
126.470480704295
제주시 구좌읍 김녕로14길 6
33.5543515285666
126.751328022367
제주특별자치도 제주시 한림읍 한림리 966-1번지 (한림리)
33.4124054749497
126.269467817541


In [14]:
lat, long
len(long), len(df_clinic)

(11, 11)

In [15]:
# 데이타프레임에 위도, 경도 컬럼 추가 
df_clinic['위도']=lat
df_clinic['경도']=long
df_clinic

,시군구,주소,위도,경도
의료기관명,,,,
서귀포시(서귀포보건소),서귀포시,제주 서귀포시 서홍동 447-3,33.2532700888151,126.555250836738
서귀포열린병원,서귀포시,제주특별자치도 서귀포시 일주동로 8638,33.2544659162544,126.565425372675
서귀포의료원,서귀포시,제주특별자치도 서귀포시 장수로 47,33.2559809912099,126.562948872895
서귀포시(서부보건소),서귀포시,제주 서귀포시 대정읍 하모리 767-8 서부보건소,33.2186942969756,126.253410446884
제주시제주보건소,제주시,제주 제주시 도남동 567-1,33.4917394006488,126.520234177096
제주한라병원,제주시,도령로 65,33.4902661749243,126.484919722369
한마음병원,제주시,연신로 52,33.4962868079834,126.546238571767
한국병원,제주시,서광로 193,33.5003915262127,126.516729196278
중앙병원,제주시,월랑로 91,33.4923211704292,126.470480704295


In [16]:
# 위도, 경도의 자료형은? 
df_clinic.dtypes

시군구    object
주소     object
위도     object
경도     object
dtype: object

In [17]:
# 자료형 변경 
df_clinic['위도'] = df_clinic['위도'].astype(float)
df_clinic['경도'] = df_clinic['경도'].astype(float)
df_clinic.dtypes

시군구     object
주소      object
위도     float64
경도     float64
dtype: object

In [18]:
# 진료소의 위도, 경도의 평균 
lat_center = df_clinic['위도'].mean()
long_center = df_clinic['경도'].mean()
lat_center, long_center

(33.40183394336088, 126.49967579462773)

In [19]:
# 지도 시작 중앙값 설정 
map = folium.Map(location=[lat_center, long_center], 
                 zoom_start=10
                )

for i in range(len(df_clinic)):
    folium.Marker([df_clinic.iloc[i,2], \
                   df_clinic.iloc[i,3]], \
                   popup=df_clinic.index[i], \
                   icon=folium.Icon(icon='bookmark', \
                                    color='red',  \
                                    prefix='fa')\
                 ).add_to(map)
map

### 주소가 잘못된 경우 return 값은?

In [20]:
address='서울 강동구 성내3, 4동 541-2'


# 퀴즈 - 경기도 선별 진료소의 지도 시각화 